In [1]:
import sqlite3
con = sqlite3.connect('db')

In [2]:
import pandas as pd

In [3]:
sqlite3.sqlite_version

'3.39.4'

In [4]:
def select(sql):
    return pd.read_sql(sql, con)

In [5]:
t = pd.DataFrame({'id':[1,1,2,2,3],
                  'name':['a','b','c','d','e']})

In [6]:
t

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [8]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

5

In [9]:
sql = '''select * from dupl_test t'''

In [10]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [11]:
sql = '''select t.id from dupl_test t
group by t.id
having count(1) > 1'''

In [12]:
select(sql)

,id
0,1
1,2


In [13]:
sql = '''select * from dupl_test t
where t.id in (select t.id from dupl_test t
group by t.id
having count(1) > 1)'''

In [14]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [15]:
sql = '''
drop table if exists dupls;
create table dupls as
select t.id from dupl_test t
group by t.id
having count(1) > 1'''

In [17]:
cur = con.cursor()

In [18]:
cur.executescript(sql)

In [19]:
sql = '''select * from dupls t'''

In [20]:
select(sql)

,id
0,1
1,2


In [21]:
sql = '''
select * from

(select t.id,count(1) as cnt from dupl_test t
group by t.id) t

where t.cnt > 1
'''

In [22]:
select(sql)

,id,cnt
0,1,2
1,2,2


In [23]:
"CTE (with)"

'CTE (with)'

In [24]:
sql = '''
select * from (

select * from

(select t.id,count(1) as cnt from dupl_test t
group by t.id) t

where t.cnt > 1) t

where t.id = 1
'''

In [25]:
select(sql)

,id,cnt
0,1,2


In [26]:
sql = '''
with id_cnt as (
select t.id,count(1) as cnt from dupl_test t
group by t.id),

id_cnt_2 as

(select * from id_cnt t
where t.cnt > 1)

select * from id_cnt_2 t
where t.id = 1

'''

In [27]:
select(sql)

,id,cnt
0,1,2
